In [1]:
!pip install --upgrade pip
!pip install numpy==1.24.4
!pip install pandas-ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 22.0 MB/s  0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
ERROR: Failed to build 'numpy' when getting requirements to build wheel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 18.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 54.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 117.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 135.1 MB/s  0:00

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import pandas_ta as ta
from pandas import read_csv
from pandas.core.frame import DataFrame
#from tensorflow.keras.models import Model, Sequential,load_model
#import tensorflow as tf
import os
import numpy as np
from numpy import split, array, dstack
import math
from math import sqrt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Flatten, Dropout
from keras.layers import LSTM , GRU
from keras.optimizers import Adam
import csv
from scipy.stats import zscore
from datetime import datetime
import pickle
from keras.models import load_model
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

## Functions

### Company data feature engineering functions

In [3]:
# fill missing values with a value at the same time one day ago
def fill_missing2(values):
 one_day = 1
 for row in range(values.shape[0]):
  for col in range(values.shape[1]):
    if np.isnan(values[row, col]):
     values[row, col] = values[row - one_day, col]

def fill_missing(values):
  return pd.DataFrame(values).fillna(method='ffill').values

def normalized_df(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#def zlema(src, length):
#    lag = round((length - 1) / 2)
#    ema_data = src + (src - src[lag])
#    zl= ta.ema(ema_data,length = 5)
#    return zl


In [4]:
def get_company_df(dataset,company_name,start_date):
  company = dataset[(dataset['trading_name '] == company_name)]
  new_date_range = pd.date_range(start=start_date, end="2020-03-05", freq="D")
  company = company.reindex(new_date_range, fill_value=np.nan)
  company = company.drop(["symbol",'name','trading_name ','sectoer'], axis='columns')
  company.replace('?', np.nan, inplace=True)
  company = company.astype('float32')
  company[:] = fill_missing(company.values)
  company['zg'],company['sg'],company['xg'],company['atr'] = ta.aberration(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['lower'],company['mid'],company['upper'] = ta.accbands(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ad'] = ta.ad(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['adosc'] = ta.adosc(high=company['high'],low= company['low'],volume=company['volume_traded '],close=company['close'],fast=1,slow=5)
  # company['adx'],company['dmp'],company['dmn'] = ta.adx(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['alma'] = ta.alma(company['close'],length=5)
  company['amat1'], company['amat2']= ta.amat(company['close'],length=5)
  # company['ao'] = ta.ao(high=company['high'],low= company['low'],close=company['close'],fast=1,slow=5)
  company['aobv1'],company['aobv2'],company['aobv3'],company['aobv4'],company['aobv5'],company['aobv6'],company['aobv7'] = ta.aobv(company['close'],company['volume_traded '])
  # company['apo'] = ta.apo(company['close'],fast=1,slow=5)
  company['aroon_up'],company['aroon_down'],company['aroon_osc'] = ta.aroon(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['atr']= ta.atr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['bbands_lower'],company['bbands_mid'],company['bbands_upper'],company['bbands_bandwidth'],company['bbands_percent'] = ta.bbands(company['close'],length=5)
  company['bias'] = ta.bias(company['close'],length=5)
  company['bop']= ta.bop(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['ar'],company['br']= ta.brar(open_ = company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cci']= ta.cci(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cfo'] = ta.cfo(company['close'],length=5)
  company['cg'] = ta.cg(company['close'],length=5)
  company['chop']= ta.chop(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['cksp_long'], company['cksp_short']= ta.cksp(high=company['high'],low= company['low'],close=company['close'])
  company['cmf']= ta.cmf(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],open = company['open'],length=5)
  company['cmo'] = ta.cmo(company['close'],length=5)
  company['coppock'] = ta.coppock(company['close'],length=5,fast=6,slow=11)
  company['cti'] = ta.cti(company['close'],length=5)
  company['decay'] = ta.decay(company['close'],length=5)
  company['decreasing'] = ta.decreasing(company['close'],length=5)
  company['dema'] = ta.dema(company['close'],length=5)
  company['dmp'],company['dmn']= ta.dm(high=company['high'],low= company['low'],length=5)
  company['doncian_lower'],company['doncian_mid'],company['doncian_upper']= ta.donchian(high=company['high'],low= company['low'],lower_length=5,upper_length=5)
  company['dpo'] = ta.dpo(company['close'],length=5)
  company['ebsw'] = ta.ebsw(company['close'],length=5)
  company['efi'] = ta.efi(company['close'],company['volume_traded '],length=5)
  company['ema'] = ta.ema(company['close'],length=5)
  company['entropy'] = ta.entropy(company['close'],length=5)
  company['eom']= ta.eom(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '],length=5)
  company['er'] = ta.er(company['close'],length=5)
  company['eri_bull'],company['eri_bear'] = ta.eri(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['fisher1'],company['fisher2'] = ta.fisher(high=company['high'],low= company['low'],length=5)
  company['fwma'] = ta.fwma(company['close'],length=5)
  company['hilo'],company['hilol'],company['hilos']=ta.hilo(close=company['close'],high=company['high'],low= company['low'],high_length=5,low_length=5)
  company['hma'] = ta.hma(company['close'],length=5)
  company['hl2'] = ta.hl2(company['high'],company['low'])
  company['hlc3'] = ta.hlc3(company['high'],company['low'],company['close'])
  # company['hwm'],company['hwu'],company['hwl'] = ta.hwc(company['close'])
  company['hwma '] = ta.dema(company['close'])
  company['increasing'] = ta.increasing(company['close'],length=5)
  company['inertia'] = ta.inertia(high=company['high'],low= company['low'],close=company['close'],open = company['open'],rvi_length=5,length=5)
  company['jma'] = ta.jma(company['close'],length=5)
  company['kama'] = ta.kama(company['close'],length=5)
  company['kcl'],company['kcb'],company['kcu']= ta.kc(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kdj1'],company['kdj2'],company['kdj3'] = ta.kdj(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['kurtosis'] = ta.kurtosis(company['close'],length=5)
  company['kvo'],company['kvo_s'] = ta.kvo(high=company['high'],low= company['low'],close=company['close'],volume = company['volume_traded '])
  company['linreg'] = ta.linreg(company['close'],length=5)
  company['log_return'] = ta.log_return(company['close'],length=5)
  company['macd'],company['macd_hist'],company['macd_sig'] = ta.macd(company['close'])
  company['mad'] = ta.mad(company['close'],length=5)
  company['massi'] = ta.massi(high=company['high'],low= company['low'],fast=1,slow=5)
  company['median'] = ta.median(company['close'],length=5)
  company['mfi'] = ta.mfi(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '],length=5)
  company['midpoint'] = ta.midpoint(company['close'],length=5)
  company['midprice'] = ta.midprice(high=company['high'],low= company['low'],length=5)
  company['mom'] = ta.mom(company['close'],length=5)
  company['natr'] = ta.natr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['nvi'] = ta.nvi(company['close'],company['volume_traded '],length=5)
  company['obv'] = ta.obv(company['close'],company['volume_traded '])
  company['pdist'] = ta.pdist(open_=company['open'],high=company['high'],low= company['low'],close=company['close'])
  company['percent_return'] = ta.percent_return(company['close'],length=5)
  company['pgo'] = ta.pgo(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['psl'] = ta.psl(company['close'],company['open'],length=5)
  # company['pvi'] = ta.pvi(company['close'],company['volume_traded '],length=5)
  company['pvol'] = ta.pvol(company['close'],company['volume_traded '])
  company['pvt'] = ta.pvt(company['close'],company['volume_traded '])
  company['pwma'] = ta.pwma(company['close'],length=5)
  company['qqe'],company['rsi_ma'] ,company['qqel'] ,company['qqes']  = ta.qqe(company['close'],length=5)
  company['qstick'] = ta.qstick(company['open'],company['close'],length=5)
  company['quantile'] = ta.quantile(company['close'],length=5)
  company['rma'] = ta.rma(company['close'],length=5)
  company['roc'] = ta.roc(company['close'],length=5)
  company['rsi'] = ta.rsi(company['close'],length=5)
  company['rsx'] = ta.rsx(company['close'],length=5)
  # company['rvgi1'],company['rvgi2'] = ta.rvgi(open_=company['open'],high=company['high'],low= company['low'],close=company['close'],length=5)
  #company['rvil'],company['rvib'],company['rviu'],company['rvi1'],company['rvi2'],company['rvi3'],company['rvi4'],company['rvi5'],company['rvi6'],company['rvi7'],company['rvi8'],company['rvi9'],company['rvi10'],company['rvi11'] = ta.rvi(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['sinwma'] = ta.sinwma(company['close'],length=5)
  company['skew'] = ta.skew(company['close'],length=5)
  company['slope'] = ta.slope(company['close'],length=5)
  company['sma'] = ta.sma(company['close'],length=5)
  company['smi'],company['smi_s'],company['smi_osc'] = ta.smi(company['close'])
  company['ssf'] = ta.ssf(company['close'],length=5)
  company['stc'],company['stc_macd'],company['stc_stoch'] = ta.stc(company['close'],tclength=5)
  company['stdev'] = ta.stdev(company['close'],length=5)
  # company['stoch_k'],company['stoch_d'] = ta.momentum.stoch(high=company['high'],low= company['low'],close=company['close'])
  company['stochrsi_k'],company['stochrsi_d']= ta.stochrsi(company['close'],length=5)
  company['swma'] = ta.swma(company['close'],length=5)
  company['t3'] = ta.t3(company['close'],length=5)
  company['tema'] = ta.tema(company['close'],length=5)
  company['trima'] = ta.trima(company['close'],length=5)
  company['trix1'],company['trix2']  = ta.trix(company['close'],length=5)
  company['true_range'] = ta.true_range(high=company['high'],low= company['low'],close=company['close'])
  company['tema'] = ta.tema(company['close'],length=5)
  company['ui'] = ta.ui(high=company['high'],close =company['close'],length=5)
  company['uo'] = ta.uo(company['high'],company['low'],company['close'],fast=1,medium=3,slow=5)
  company['variance'] = ta.variance(company['close'],length=5)
  company['vhf'] = ta.vhf(company['close'],length=5)
  company['vidya'] = ta.vidya(company['close'],length=5)
  company['vortex1'],company['vortex2'] = ta.vortex(company['high'],company['low'],company['close'],length=5)
  # company['vp1'],company['vp2'],company['vp3'],company['vp4'],company['vp5'],company['vp6'] = ta.vp(company['close'],company['volume_traded '])
  company['vwap'] = ta.vwap(high=company['high'],low= company['low'],close=company['close'],volume=company['volume_traded '])
  company['vmwa'] = ta.vwma(volume=company['volume_traded '],close=company['close'],length=5)
  company['wcp'] = ta.wcp(high=company['high'],low= company['low'],close=company['close'])
  company['willr'] = ta.willr(high=company['high'],low= company['low'],close=company['close'],length=5)
  company['wma'] = ta.wma(company['close'],length=5)
  company['zlma'] = ta.zlma(company['close'],length=5)
  company['zscore'] = ta.zscore(company['close'],length=5)
  #company['zlema'] = zlema(company['close'],length=5)
  company = company.fillna(0)
  #selected features based on corrlation sheet
  selected_ti = ['open','high','low','close','alma','decay','dema','ema','fwma','hma','hl2','hlc3','hwma ','jma','kama','linreg','median','midpoint','midprice','pwma','quantile','rma','sinwma','sma','ssf','swma','t3','tema','trima','vwap','vmwa','wcp','wma','zlma']
  new_company_df =  company[selected_ti]
  return new_company_df, selected_ti

### Company data prepration functions

In [6]:
# convert history into inputs and outputs
def to_supervised(X, y, n_input, n_output):
    Xs, ys = [], []
    for i in range(len(X) - n_input - n_output + 1):
        Xs.append(X.iloc[i:i+n_input].values)
        ys.append(y.iloc[i+n_input:i+n_input+n_output].values)
    return np.array(Xs), np.array(ys)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [7]:
  def prepare_company_for_training_and_testing(company, selected_ti, target_col = 'close',
                                              n_input=5, n_output=5):

    # ===============================
    # 1. Split data temporally
    # ===============================
    train = company.loc[:'2015-12-31']
    val   = company.loc['2016-01-01':'2016-12-31']
    test  = company.loc['2017-01-01':]

    print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')

    # ===============================
    # 2. Feature scaler (X)
    # ===============================
    feature_scaler = StandardScaler()
    feature_scaler.fit(train[selected_ti])

    train_x_scaled = DataFrame(
        feature_scaler.transform(train[selected_ti]),
        columns=selected_ti,
        index=train.index
    )

    val_x_scaled = DataFrame(
        feature_scaler.transform(val[selected_ti]),
        columns=selected_ti,
        index=val.index
    )

    test_x_scaled = DataFrame(
        feature_scaler.transform(test[selected_ti]),
        columns=selected_ti,
        index=test.index
    )

    # ===============================
    # 3. Target scaler (y)
    # ===============================
    global target_scaler
    target_scaler = StandardScaler()
    target_scaler.fit(train[[target_col]])

    train_y_scaled = DataFrame(
        target_scaler.transform(train[[target_col]]),
        columns=[target_col],
        index=train.index
    )

    val_y_scaled = DataFrame(
        target_scaler.transform(val[[target_col]]),
        columns=[target_col],
        index=val.index
    )

    test_y_scaled = DataFrame(
        target_scaler.transform(test[[target_col]]),
        columns=[target_col],
        index=test.index
    )

    print("*******************************************")
    print("Train data after multistep approach")

    # ===============================
    # 4. Supervised transformation
    # ===============================
    train_x, train_y = to_supervised(train_x_scaled, train_y_scaled,
                                     n_input, n_output)

    print("Validation multistep approach")
    val_x, val_y = to_supervised(val_x_scaled, val_y_scaled,
                                 n_input, n_output)

    print("Test multistep approach")
    test_x, test_y = to_supervised(test_x_scaled, test_y_scaled,
                                   n_input, n_output)

    return (
        train_x, train_y,
        val_x, val_y,
        test_x, test_y,
        test,
    )

### Company prdiction prepration for ensemble function

In [14]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  for i in range(len(y_true) - n_input - n_out + 1):
      y.append(y_true.iloc[i+n_input:i+n_input+n_out].values)
  return np.array(y)

# make a forecast
def forecast(model, history):
  yhat = model.predict(history, verbose=0)
  return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  predicted_not_normlized = target_scaler.inverse_transform(predicted)
  predicted_not_normlized_rounded = [np.round(num,2) for num in predicted_not_normlized]
  print("Actual",actual[1000])
  print("Predicted_not_normlized",predicted_not_normlized_rounded[1000])
  mse_score= mse(actual,predicted_not_normlized_rounded)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized_rounded)
  final_r2_score = r2_score(actual,predicted_not_normlized_rounded,multioutput='variance_weighted')
  return mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized_rounded

# summarize scores
def summarize_scores(name, mse_score, rmse_score,mae_score,final_r2_score):
  #s_scores = ', '.join(['%.1f' % s for s in scores])
  #print('%s: [%.3f] %s' % (name, score, s_scores))
  print(f'*************{name}**********')
  print("MAE: ",mae_score)
  print("MSE: ",mse_score)
  print("RMSE: ",rmse_score)
  print("R2: ",final_r2_score)

# evaluate a single model
def evaluate_model(test_x,test_y,model):
	# walk-forward validation over each week
  predictions = list()
  predictions = forecast(model, test_x)
  # evaluate predictions days for each week
  predictions = array(predictions)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

# def load_all_models(n_models,company_name):
#   import pickle
#   all_models = list()
#   all_models_orgnized = list()
#   dirname = '/content/drive/MyDrive/KFUPM/Graduation/best_models/'+company_name
#   for filename in os.listdir(dirname):
#     root, ext = os.path.splitext(filename)
#     if root.startswith(str(company_name)) and ext == '.pkl':
#       with open(dirname+'/'+root+ext,'rb') as file:
#         print(dirname+'/'+root+ext)
#         model = pickle.load(file)

#       #model_path = os.path.join(dirname,filename)
#       # print(model_path)
#       # model =tf.keras.models.load_model(model_path)
#       all_models.append(model)
#   return all_models

def load_all_models(n_models, company_name):
    all_models = []

    # Define the order of the models
    model_order = ["BRNN", "GRU", "LSTM"]

    dirname = '/content/drive/MyDrive/KFUPM/Graduation/final_experiments/best_models/' + company_name

    for filename in os.listdir(dirname):
        root, ext = os.path.splitext(filename)
        if root.startswith(str(company_name)) and ext == '.pkl':
            with open(os.path.join(dirname, filename), 'rb') as file:
                #print(os.path.join(dirname, filename))
                model = pickle.load(file)
                all_models.append((root, model))  # Store the root and model together

    # Sort the models according to the predefined order
    all_models_sorted = sorted(all_models, key=lambda x: model_order.index(x[0].split('_')[-2]))
    print("**********Models Order***************")
    [print(x[0]) for x in all_models_sorted]
    print("*************************************")
    # Extract the sorted models
    all_models_sorted = [model for _, model in all_models_sorted]

    return all_models_sorted

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat #
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	#stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

def stacking_process(dataset,selected_companies_dic,company_name):
  #Get company and selected TIs
  company, company_selected_ti = get_company_df(dataset,selected_companies_dic[company_name][0],selected_companies_dic[company_name][1])
  train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(company, company_selected_ti)
  n_members = 3
  members = load_all_models(n_members,company_name)
  print('Loaded %d models' % len(members))
  # # ANN
  # train_eval_stacking_model(company_name,members,len(members),train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU", "LSTM"])
  # train_eval_stacking_model(company_name,[members[0],members[1]] ,len(members)-1, train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU"])
  # train_eval_stacking_model(company_name,[members[0],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "LSTM"])
  # train_eval_stacking_model(company_name,[members[1],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["GRU", "LSTM"])
  # # LSTM
  # train_eval_stacking_model_lstm(company_name,members,len(members),train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU", "LSTM"])
  # train_eval_stacking_model_lstm(company_name,[members[0],members[1]] ,len(members)-1, train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU"])
  # train_eval_stacking_model_lstm(company_name,[members[0],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "LSTM"])
  # train_eval_stacking_model_lstm(company_name,[members[1],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["GRU", "LSTM"])
  # # GRU
  # train_eval_stacking_model_gru(company_name,members,len(members),train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU", "LSTM"])
  # train_eval_stacking_model_gru(company_name,[members[0],members[1]] ,len(members)-1, train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU"])
  # train_eval_stacking_model_gru(company_name,[members[0],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "LSTM"])
  # train_eval_stacking_model_gru(company_name,[members[1],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["GRU", "LSTM"])
  # BiRNN
  train_eval_stacking_model_birnn(company_name,members,len(members),train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU", "LSTM"])
  train_eval_stacking_model_birnn(company_name,[members[0],members[1]] ,len(members)-1, train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "GRU"])
  train_eval_stacking_model_birnn(company_name,[members[0],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["BRNN", "LSTM"])
  train_eval_stacking_model_birnn(company_name,[members[1],members[2]],len(members)-1,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,["GRU", "LSTM"])

  # Define the model
def train_eval_stacking_model(company_name,members,members_num,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,models_names):
  # Shape: (samples, timesteps, features)
  stacked_train = stacked_dataset(members, train_x_normlized)
  stacked_val = stacked_dataset(members, val_x_normlized)
  stacked_test = stacked_dataset(members, test_x_normlized)
  model = Sequential()
  # Input layer - Flatten the (timesteps, features) into a 1D vector
  model.add(Flatten(input_shape=(5, members_num)))  # 5 timesteps and 3 features per timestep
  # Hidden layers
  model.add(Dense(128, activation='relu'))  # First hidden layer
  model.add(Dense(64, activation='relu'))  # First hidden layer
  model.add(Dense(32, activation='relu'))  # Second hidden layer
  # Output layer - Regression example (single output for simplicity)
  model.add(Dense(5))  # Adjust the output size based on your specific task
  # Compile the model
  model.compile(optimizer='adam', loss='mse')
  # Model summary
  model.summary()
  model.fit(
    stacked_train,
    train_y_normlized,
    epochs=100,
    validation_data=(stacked_val,val_y_normlized))
  predictions_normlized = model.predict(stacked_test)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,3], 5, 5)
  mse_score, rmse_score,mae_score,final_r2_score,predictions = evaluate_model(stacked_test,test_y_for_evaluation,model)
  separator = '_'  # Using a dash as the separator
  models_names_str = separator.join(models_names)
  pickle.dump(model , open(f'/content/drive/MyDrive/KFUPM/Graduation/best_models/{company_name}/{models_names_str}_meta_learner_ann' , 'wb'))
  summarize_scores(f'Prposed ANN of {str(models_names)}', mse_score, rmse_score,mae_score,final_r2_score)

    # Define the model
def train_eval_stacking_model_lstm(company_name,members,members_num,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,models_names):
  # Shape: (samples, timesteps, features)
  n_timesteps = 5
  stacked_train = stacked_dataset(members, train_x_normlized)
  stacked_val = stacked_dataset(members, val_x_normlized)
  stacked_test = stacked_dataset(members, test_x_normlized)
  # Define the model
  model = Sequential()
  model.add(LSTM(128, activation='relu', input_shape=(n_timesteps, members_num)))
  model.add(Dense(n_timesteps))  # n_outputs is the number of steps to forecast
  model.compile(optimizer='adam', loss='mse')
  # Compile the model
  # Model summary
  model.summary()
  model.fit(
    stacked_train,
    train_y_normlized,
    epochs=100,
    validation_data=(stacked_val,val_y_normlized))
  predictions_normlized = model.predict(stacked_test)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,3], 5, 5)
  mse_score, rmse_score,mae_score,final_r2_score,predictions = evaluate_model(stacked_test,test_y_for_evaluation,model)
  separator = '_'  # Using a dash as the separator
  models_names_str = separator.join(models_names)
  pickle.dump(model , open(f'/content/drive/MyDrive/KFUPM/Graduation/best_models/{company_name}/{models_names_str}_meta_learner_lstm' , 'wb'))
  summarize_scores(f'Prposed LSTM of {str(models_names)}', mse_score, rmse_score,mae_score,final_r2_score)

def train_eval_stacking_model_gru(company_name,members,members_num,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,models_names):
  # Shape: (samples, timesteps, features)
  n_timesteps = 5
  stacked_train = stacked_dataset(members, train_x_normlized)
  stacked_val = stacked_dataset(members, val_x_normlized)
  stacked_test = stacked_dataset(members, test_x_normlized)
  # Define the model
  model = Sequential()
  model.add(GRU(128, activation='relu', input_shape=(n_timesteps, members_num)))
  model.add(Dense(n_timesteps))  # n_outputs is the number of steps to forecast
  model.compile(optimizer='adam', loss='mse')
  # Compile the model
  # Model summary
  model.summary()
  model.fit(
    stacked_train,
    train_y_normlized,
    epochs=100,
    validation_data=(stacked_val,val_y_normlized))
  predictions_normlized = model.predict(stacked_test)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,3], 5, 5)
  mse_score, rmse_score,mae_score,final_r2_score,predictions = evaluate_model(stacked_test,test_y_for_evaluation,model)
  separator = '_'  # Using a dash as the separator
  models_names_str = separator.join(models_names)
  pickle.dump(model , open(f'/content/drive/MyDrive/KFUPM/Graduation/best_models/{company_name}/{models_names_str}_meta_learner_gru' , 'wb'))
  summarize_scores(f'Prposed GRU of {str(models_names)}', mse_score, rmse_score,mae_score,final_r2_score)

def train_eval_stacking_model_birnn(company_name,members,members_num,train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,models_names):
  # Shape: (samples, timesteps, features)
  n_timesteps = 5
  stacked_train = stacked_dataset(members, train_x_normlized)
  stacked_val = stacked_dataset(members, val_x_normlized)
  stacked_test = stacked_dataset(members, test_x_normlized)
  # Define the model
  model = Sequential()
  model.add(BRNN(128, activation='relu', input_shape=(n_timesteps, members_num)))
  model.add(Dense(n_timesteps))  # n_outputs is the number of steps to forecast
  model.compile(optimizer='adam', loss='mse')
  # Compile the model
  # Model summary
  model.summary()
  model.fit(
    stacked_train,
    train_y_normlized,
    epochs=100,
    validation_data=(stacked_val,val_y_normlized))
  predictions_normlized = model.predict(stacked_test)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,3], 5, 5)
  mse_score, rmse_score,mae_score,final_r2_score,predictions = evaluate_model(stacked_test,test_y_for_evaluation,model)
  separator = '_'  # Using a dash as the separator
  models_names_str = separator.join(models_names)
  pickle.dump(model , open(f'/content/drive/MyDrive/KFUPM/Graduation/best_models/{company_name}/{models_names_str}_meta_learner_gru' , 'wb'))
  summarize_scores(f'Prposed GRU of {str(models_names)}', mse_score, rmse_score,mae_score,final_r2_score)


def averaging_process(dataset,selected_companies_dic,company_name):
  #Get company and selected TIs
  company, company_selected_ti = get_company_df(dataset,selected_companies_dic[company_name][0],selected_companies_dic[company_name][1])
  train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(company, company_selected_ti)
  n_members = 3
  members = load_all_models(n_members,company_name)
  print('Loaded %d models' % len(members))
  stacked_train = stacked_dataset(members, train_x_normlized)
  stacked_val = stacked_dataset(members, val_x_normlized)
  stacked_test_3 = stacked_dataset(members, test_x_normlized)
  means_stacked_test_3 = np.mean(stacked_test_3, axis=2)
  stacked_test_b_g = stacked_dataset([members[0],members[1]], test_x_normlized)
  means_stacked_test_b_g = np.mean(stacked_test_b_g, axis=2)
  stacked_test_b_l = stacked_dataset([members[0],members[2]], test_x_normlized)
  means_stacked_test_b_l = np.mean(stacked_test_b_l, axis=2)
  stacked_test_g_l = stacked_dataset([members[1],members[2]], test_x_normlized)
  means_stacked_test_g_l = np.mean(stacked_test_g_l, axis=2)
  test_y_for_evaluation = true_y_to_evaluation_form(test.iloc[:,3], 5, 5)
  mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized = evaluate_forecasts(test_y_for_evaluation, means_stacked_test_3)
  summarize_scores('Prposed Averaging (BRNN,GRU,LSTM) Ensemble', mse_score, rmse_score,mae_score,final_r2_score)
  mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized = evaluate_forecasts(test_y_for_evaluation, means_stacked_test_b_g)
  summarize_scores('Prposed Averaging (BRNN,GRU) Ensemble', mse_score, rmse_score,mae_score,final_r2_score)
  mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized = evaluate_forecasts(test_y_for_evaluation, means_stacked_test_b_l)
  summarize_scores('Prposed Averaging (BRNN,LSTM) Ensemble', mse_score, rmse_score,mae_score,final_r2_score)
  mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized = evaluate_forecasts(test_y_for_evaluation, means_stacked_test_g_l)
  summarize_scores('Prposed Averaging (GRU,LSTM) Ensemble', mse_score, rmse_score,mae_score,final_r2_score)




## Excution

In [10]:
dataset = pd.read_csv("/content/drive/MyDrive/KFUPM/Graduation/data/Saudi/Tadawul_stcks.csv",index_col='date',parse_dates=True)
dataset.sort_values(by='date', inplace = True)

In [11]:
dataset.head()

,symbol,name,trading_name,sectoer,open,high,low,close,change,perc_Change,volume_traded,value_traded,no_trades
date,,,,,,,,,,,,,
2001-12-31,2050,Savola Group,SAVOLA GROUP,Consumer Staples,5.25,5.50,5.25,5.25,5.25,0.00,1077466.0,5741161.25,47.0
2001-12-31,2110,Saudi Cable Co.,SAUDI CABLE,Industrials,27.76,27.76,27.76,27.76,-24.99,-47.37,27227.0,754050.00,18.0
2001-12-31,4150,Arriyadh Development Co.,ARDCO,Real Estate,3.94,3.94,3.94,3.94,-31.56,-88.90,1667214.0,6634241.75,124.0
2001-12-31,4010,Dur Hospitality Co.,DUR,Consumer Discretionary,9.50,9.50,9.30,9.30,9.30,0.00,40524.0,381406.50,7.0
2001-12-31,1060,Saudi British Bank,SABB,Financials,8.90,8.90,8.90,8.90,-323.10,-97.32,516587.0,4613774.75,21.0


In [12]:
selected_companies_dic = {"ADC":["ADC","2001-12-31"], "ALRAJHI":["ALRAJHI","2001-12-31"], "GACO":["GACO","2002-01-01"], "EMAAR EC":["EMAAR EC","2006-10-06"], "FITAIHI GROUP":["FITAIHI GROUP","2002-01-26"], "SARCO":["SARCO","2002-01-23"], "SAUDI ELECTRICITY":["SAUDI ELECTRICITY","2002-06-08"], "SPIMACO":["SPIMACO","2001-12-31"], "STC":["STC","2003-01-25"], "TASNEE":["TASNEE","2001-12-31"] }

### Avergaing ensemble

In [ ]:
averaging_process(dataset,selected_companies_dic,"ADC")

In [ ]:
averaging_process(dataset,selected_companies_dic,"ALRAJHI")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
ALRAJHI_BRNN_900
ALRAJHI_GRU_800
ALRAJHI_LSTM_400
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.7400903
MSE:  1.1457332
RMSE:  1.0703892923915768
R2:  0.989652156829834
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.74094236
MSE:  1.1409229
RMSE:  1.068139927169932
R2:  0.9896956086158752
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.74532765
MSE:  1.1544065
RMSE:  1.0744331284665354
R2:  0.9895738363265991
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.74542934
MSE:  1.1662134
R

In [ ]:
averaging_process(dataset,selected_companies_dic,"GACO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5113, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5103, 5, 34)
y shape: (5103, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
GACO_BRNN_900
GACO_GRU_700
GACO_LSTM_1000
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.17309861
MSE:  0.069779634
RMSE:  0.2641583511375478
R2:  0.9602474570274353
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.17267756
MSE:  0.06974802
RMSE:  0.26409850749717473
R2:  0.9602655172348022
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.17372598
MSE:  0.06976582
RMSE:  0.2641322038284641
R2:  0.9602553844451904
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.17440668
MSE:  0.070549704

In [ ]:
averaging_process(dataset,selected_companies_dic,"EMAAR EC")

/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-

Train shape: (3374, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (3364, 5, 34)
y shape: (3364, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
EMAAR EC_BRNN_1024
EMAAR EC_GRU_600
EMAAR EC_LSTM_1000
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.21529675
MSE:  0.106433846
RMSE:  0.32624200535256237
R2:  0.9874013662338257
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.21539669
MSE:  0.106215075
RMSE:  0.3259065428284524
R2:  0.9874273538589478
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.21983957
MSE:  0.10867927
RMSE:  0.32966539435378994
R2:  0.9871355891227722
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.21529837
MS

In [ ]:
averaging_process(dataset,selected_companies_dic,"FITAIHI GROUP")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5088, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5078, 5, 34)
y shape: (5078, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
FITAIHI GROUP_BRNN_200
FITAIHI GROUP_GRU_512
FITAIHI GROUP_LSTM_900
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.12593569
MSE:  0.03647279
RMSE:  0.1909785064954123
R2:  0.9490616321563721
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.12736279
MSE:  0.036679614
RMSE:  0.19151922706427815
R2:  0.9487727284431458
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.12671553
MSE:  0.03687301
RMSE:  0.19202345964973366
R2:  0.9485026001930237
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0

In [ ]:
averaging_process(dataset,selected_companies_dic,"SARCO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tre

Train shape: (5091, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5081, 5, 34)
y shape: (5081, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SARCO_BRNN_500
SARCO_GRU_800
SARCO_LSTM_1024
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.75871354
MSE:  1.5187334
RMSE:  1.23236901219766
R2:  0.9381060004234314
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.7603718
MSE:  1.5144908
RMSE:  1.2306465141620537
R2:  0.9382789731025696
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.76340216
MSE:  1.514319
RMSE:  1.230576670924523
R2:  0.9382859468460083
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.77046454
MSE:  1.5549471
RMSE:  1

In [ ]:
averaging_process(dataset,selected_companies_dic,"SAUDI ELECTRICITY")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tre

Train shape: (4955, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4945, 5, 34)
y shape: (4945, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SAUDI ELECTRICITY_BRNN_300
SAUDI ELECTRICITY_GRU_243
SAUDI ELECTRICITY_LSTM_1024
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.3084051
MSE:  0.21846446
RMSE:  0.46740182279191367
R2:  0.9748988747596741
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.30826384
MSE:  0.21800585
RMSE:  0.4669109667925358
R2:  0.9749515056610107
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.31104514
MSE:  0.22023132
RMSE:  0.46928810387040537
R2:  0.9746958613395691
*************Prposed Averaging (GRU,LSTM) Ensemble*******

In [ ]:
averaging_process(dataset,selected_companies_dic,"SPIMACO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SPIMACO_BRNN_900
SPIMACO_GRU_256
SPIMACO_LSTM_600
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.34916484
MSE:  0.27027065
RMSE:  0.5198756059082199
R2:  0.9778169393539429
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.34838063
MSE:  0.26960796
RMSE:  0.5192378657010926
R2:  0.9778713583946228
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.35168386
MSE:  0.27407718
RMSE:  0.5235238075271549
R2:  0.977504551410675
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.35388014
MSE:  0.2734

In [ ]:
averaging_process(dataset,selected_companies_dic,"STC")

/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

Train shape: (4724, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4714, 5, 34)
y shape: (4714, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
STC_BRNN_1024
STC_GRU_729
STC_LSTM_1024
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  1.280864
MSE:  3.8261273
RMSE:  1.956048897836071
R2:  0.9816406965255737
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  1.2660339
MSE:  3.7393646
RMSE:  1.9337436810558521
R2:  0.9820570349693298
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  1.293748
MSE:  3.8837037
RMSE:  1.970711472704384
R2:  0.9813644886016846
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  1.3048966
MSE:  3.9191716
RMSE:  1.9796897

In [ ]:
averaging_process(dataset,selected_companies_dic,"TASNEE")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
TASNEE_BRNN_1024
TASNEE_GRU_512
TASNEE_LSTM_1024
*************************************
Loaded 3 models
*************Prposed Averaging (BRNN,GRU,LSTM) Ensemble**********
MAE:  0.31186083
MSE:  0.19771595
RMSE:  0.4446526205842481
R2:  0.9727327823638916
*************Prposed Averaging (BRNN,GRU) Ensemble**********
MAE:  0.30798748
MSE:  0.19497578
RMSE:  0.4415606169717841
R2:  0.9731106758117676
*************Prposed Averaging (BRNN,LSTM) Ensemble**********
MAE:  0.31522378
MSE:  0.20012769
RMSE:  0.44735633563246013
R2:  0.9724001288414001
*************Prposed Averaging (GRU,LSTM) Ensemble**********
MAE:  0.31591642
MSE:  0.200

### Stacking ensemble

In [ ]:
stacking_process(dataset,selected_companies_dic,"ADC")

In [ ]:
stacking_process(dataset,selected_companies_dic,"ALRAJHI")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-7-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
ALRAJHI_BRNN_900
ALRAJHI_GRU_800
ALRAJHI_LSTM_400
*************************************
Loaded 3 models
Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 15)                0         
                                                                 
 dense_51 (Dense)            (None, 128)               2048      
                                                                 
 dense_52 (Dense)            (None, 64)                8256      
                                             

In [ ]:
stacking_process(dataset,selected_companies_dic,"GACO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-7-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5113, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5103, 5, 34)
y shape: (5103, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
GACO_BRNN_900
GACO_GRU_700
GACO_LSTM_1000
*************************************
Loaded 3 models
Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 15)                0         
                                                                 
 dense_75 (Dense)            (None, 128)               2048      
                                                                 
 dense_76 (Dense)            (None, 64)                8256      
                                                     

In [ ]:
stacking_process(dataset,selected_companies_dic,"EMAAR EC")

/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-7-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-7-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-

Train shape: (3374, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (3364, 5, 34)
y shape: (3364, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
EMAAR EC_BRNN_1024
EMAAR EC_GRU_600
EMAAR EC_LSTM_1000
*************************************
Loaded 3 models
Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_17 (Flatten)        (None, 15)                0         
                                                                 
 dense_99 (Dense)            (None, 128)               2048      
                                                                 
 dense_100 (Dense)           (None, 64)                8256      
                                        

In [ ]:
stacking_process(dataset,selected_companies_dic,"FITAIHI GROUP")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-7-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5088, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5078, 5, 34)
y shape: (5078, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
FITAIHI GROUP_BRNN_200
FITAIHI GROUP_GRU_512
FITAIHI GROUP_LSTM_900
*************************************
Loaded 3 models
Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_21 (Flatten)        (None, 15)                0         
                                                                 
 dense_123 (Dense)           (None, 128)               2048      
                                                                 
 dense_124 (Dense)           (None, 64)                8256      
                           

In [ ]:
stacking_process(dataset,selected_companies_dic,"SARCO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tre

Train shape: (5091, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5081, 5, 34)
y shape: (5081, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SARCO_BRNN_500
SARCO_GRU_800
SARCO_LSTM_1024
*************************************
Loaded 3 models
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 15)                0         
                                                                 
 dense (Dense)               (None, 128)               2048      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                     

In [ ]:
stacking_process(dataset,selected_companies_dic,"SAUDI ELECTRICITY")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be tre

Train shape: (4955, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4945, 5, 34)
y shape: (4945, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SAUDI ELECTRICITY_BRNN_300
SAUDI ELECTRICITY_GRU_243
SAUDI ELECTRICITY_LSTM_1024
*************************************
Loaded 3 models
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 15)                0         
                                                                 
 dense_24 (Dense)            (None, 128)               2048      
                                                                 
 dense_25 (Dense)            (None, 64)                8256      
              

In [ ]:
stacking_process(dataset,selected_companies_dic,"SPIMACO")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
SPIMACO_BRNN_900
SPIMACO_GRU_256
SPIMACO_LSTM_600
*************************************
Loaded 3 models
Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 15)                0         
                                                                 
 dense_48 (Dense)            (None, 128)               2048      
                                                                 
 dense_49 (Dense)            (None, 64)                8256      
                                             

In [ ]:
stacking_process(dataset,selected_companies_dic,"STC")

/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

Train shape: (4724, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (4714, 5, 34)
y shape: (4714, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
STC_BRNN_1024
STC_GRU_729
STC_LSTM_1024
*************************************
Loaded 3 models
Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_20 (Flatten)        (None, 15)                0         
                                                                 
 dense_120 (Dense)           (None, 128)               2048      
                                                                 
 dense_121 (Dense)           (None, 64)                8256      
                                                       

In [ ]:
stacking_process(dataset,selected_companies_dic,"TASNEE")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:52: RuntimeWarning: invalid value encountered in scalar power
  rd = (divisor * (length * y2_sum - y_sum * y_sum)) ** 0.5
/usr/local/lib/python3.10/dist-packages/pandas_ta/overlap/linreg.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  return rn / rd
<ipython-input-6-81f741971fba>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company['decay'] = ta.decay(company['close'],length=5)
<ipython-input

Train shape: (5114, 34), Val shape: (366, 34), Test shape: (1160, 34)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 34)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 34)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 34)
y shape: (1150, 5)
**********Models Order***************
TASNEE_BRNN_1024
TASNEE_GRU_512
TASNEE_LSTM_1024
*************************************
Loaded 3 models
Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_16 (Flatten)        (None, 15)                0         
                                                                 
 dense_96 (Dense)            (None, 128)               2048      
                                                                 
 dense_97 (Dense)            (None, 64)                8256      
                                              